goodBadCharacterDescirptivenessClassficationWithBert(pytorch)

In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [3]:
import pandas as pd
from pandas import DataFrame

import numpy as np


data = pd.read_csv('data/character_descriptiveness_data_0_1.csv', delimiter = ',', encoding="ISO-8859-1")

data.index.name = "index"
data.columns = ["label", "text"]
data

,label,text
index,,
0,0,He was muscular with a mustache. Mrs. Dursley ...
1,0,A big hairy man was standing in the doorway. B...
2,0,I was awkward and he was really good. Although...
3,0,Mrs. Flowers looked beautiful but no one dared...
4,0,The kidsf faces were so dirty that they weren...
...,...,...
194,1,Seth was pretty much how Edwin had imagined fr...
195,1,A heroic belch of thunder followed the strange...
196,1,Raley stepped into the one-room cabin. It smel...


train_df = pd.read_csv('./nsmc/showingTelling_csv.csv', sep='\t', encoding='utf-16')
test_df = pd.read_csv('./nsmc/showingTelling_csv.txt', sep='\t', encoding='utf-16')

In [4]:
!pwd

/Users/kimkwangil/Project/01EssayFitAI/Character_analysis


In [6]:
from sklearn.model_selection import train_test_split

def shuffle(df, n=1, axis=0): #데이터가 1111 0000이라 잘 섞어주자. 
    df = data.copy()
    for _ in range(n):
        df.apply(np.random.shuffle, axis=axis)
    return df

shuffle(data)

train_df, test_df = train_test_split(data, test_size=0.2)
print(len(train_df))
print(len(test_df))

159
40


In [7]:
train_df

,label,text
index,,
196,1,Raley stepped into the one-room cabin. It smel...
101,1,A giant of a man was standing in the doorway. ...
80,0,She approached his car with a look of distaste.
18,0,I feel like I am invisible because people don...
171,1,SARAH CONNOR is not the same woman we remember...
...,...,...
19,0,He smiled at me and I felt he understood. He s...
52,0,Elrond looked ageless with dark hair and grey ...
182,1,These sharps were dressed in the heighth of fa...


In [8]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=1, random_state=999)
test_df = test_df.sample(frac=1, random_state=999)

/Users/kimkwangil/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/kimkwangil/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
train_df

,label,text
index,,
73,0,Dalton Wiseheart seemed to be a plain looking ...
25,0,Walter Elliot was very full of himself because...
149,1,"Pale hair fell in waves to his shoulders, fram..."
119,1,A black shadow dropped down into the circle. I...
29,0,She looks like she has it all but is empty ins...
...,...,...
84,0,He grew up to have a tall but weak physique.
1,0,A big hairy man was standing in the doorway. B...
82,0,The three people were very fashionable with co...


In [10]:
train_df.iloc[0, 1]

'Dalton Wiseheart seemed to be a plain looking cowboy. He was ugly with a round nose and even had a big belly.'

In [11]:
train_df.shape

(158, 2)

In [12]:
class STDataset(Dataset):
    ''' Showing Telling Corpus Dataset '''
    def __init__(self, df): #데이터 전처리
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = int(self.df.iloc[idx, 0])
        return text, label

In [13]:
ST_train_dataset = STDataset(train_df)
train_loader = DataLoader(ST_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [14]:
ST_train_dataset.__getitem__(7)

('The giant who came with Adaon was big. He has wounds on his face and much hair as well. And his neck was so thick.',
 0)

In [15]:
ST_train_dataset

In [16]:
train_loader

In [17]:
import torch

#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [18]:
for text, label in train_loader :
    print("text:",text)
    print("label:",label)

text: ('She has bright, dark eyes and satiny brown skin and stands tilted up on her toes with arms slightly extended to her sides, as if ready to take wing at the slightest sound.', 'As always, there was an all-American war hero look to him, coded in his tousled brown hair, his summer-narrowed hazel eyes, the straight nose that ancient Anglo-Saxons had graciously passed on to him. Everything about him suggested valor and power and a firm handshake.')
label: tensor([1, 1])
text: ('His heart was like a sensitive plant, that opens for a moment in the sunshine, but curls up and shrinks into itself at the slightest touch of the finger, or the lightest breath of wind.', 'Ender did not see Peter as the beautiful ten-year-old boy that grown-ups saw, with dark, tousled hair and a face that could have belonged to Alexander the Great. Ender looked at Peter only to detect anger or boredom, the dangerous moods that almost always led to pain.')
label: tensor([1, 1])
text: ('I am an invisible man. _N

In [19]:
#working properly but recomment using google colab!

optimizer = Adam(model.parameters(), lr=1e-6)

itr = 1
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0

model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1


/Users/kimkwangil/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/kimkwangil/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [20]:
# evaluation
model.eval()

nsmc_eval_dataset = STDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)
 

/Users/kimkwangil/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/Users/kimkwangil/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Test accuracy:  0.475
